In [10]:
# Import the packages
import torch
import os
import argparse
import numpy as np
import os.path as osp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from tqdm import tqdm
#from torcheval.metrics import R2Score # To be implemented
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear
from torch.nn import BatchNorm1d
import torch.optim as optim

from aux_func import*
from datset_process import *
from torch.utils.data import DataLoader

# Import the pretrained default model resnet18/resnet50/resnet101. Set pretrained=False
from torchvision.models import resnet50
import gc

In [11]:
# Define arg parser
seed=200
paser = argparse.ArgumentParser() 
args = paser.parse_args("")
np.random.seed(200)
torch.manual_seed(seed)
device=input("Enter cuda or cpu for device type")
device = torch.device(device)
#'cuda' if torch.cuda.is_available() else
device

device(type='cuda')

In [12]:
# Take user inputs 
args.dataset='miniimagenet'
args.data_path='datasets/data/mini-imagenet/'
args.num_classes=64 # By default for miniimagenet
args.image_size=84

# FSL definitions
args.num_ways=5 # Number of classes per batch
args.k_shot=10 # number of Images per class
args.query=5 # Query set of the FSL
args.unlabel=10 # Number of unlabel samples per class 
args.steps=5 # Select how many unlabeled data for each class in one iteration.
args.threshold=0.2  # Since we have 5 classes in each support set. So if all the classes are equally probable then mininmum p=0.2

# set in semi-supervised few-shot learning
num_support = args.k_shot * args.num_classes
num_query = args.query * args.num_ways
num_unlabeled = args.unlabel * args.num_ways

# Training or testing definitions 
args.episodes=600

In [13]:
# Number of sets of unlabeled data
num_select = int(args.unlabel / args.steps)

In [14]:
# Import the resnet model and define the model to be used 
model=resnet50(num_classes=args.num_classes,pretrained=False) # set pretrained=False
model.to(device)

c:\Users\kanad\Desktop\Github repos\IE_506_PROJECT\ML_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\kanad\Desktop\Github repos\IE_506_PROJECT\ML_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
def train_loop2(model1, dataset, loss_fn, optimizer, inputs, targets):
    # Define forward function
    acc =0
    def forward_fn(data, label):
        logits = model1(data)
        #logits=F.softmax(logits) # Changed here       
        loss = loss_fn(logits, label) # + entropy_loss(logits) # Combination of two loss functions used here. The entropy_loss act as regularizer
        # loss = loss_fn(logits, label)
        return loss, logits
    
    def train_step(data, label):
        optimizer.zero_grad()
        loss, logits = forward_fn(data, label)
        loss.backward()
        optimizer.step()
        return loss.item(), logits
    
    model.train()
    final_loss=0
    for i in range(len(inputs)):
        #print(i)
        img = torch.tensor(inputs[i])
        y = torch.tensor(targets[i]).long() # Changed
        img=img.unsqueeze(0).to(device)
        y=y.unsqueeze(0).to(device)
        loss, logits = train_step(img, y)
        final_loss+=loss
    final_loss/=len(inputs)
    # Check accuracy
    for i in range(len(inputs)):
        model.eval()
        #print(i)
        img = torch.tensor(inputs[i])
        y = torch.tensor(targets[i]).long() # Changed
        img=img.unsqueeze(0).to(device)
        y=y.unsqueeze(0).to(device)
        preds = model(img)
        preds = torch.argmax(preds, 1).reshape(-1)
        y = y.reshape(-1)
        if (preds==y):
          acc +=1
    acc = acc/len(inputs)*100
    return final_loss, logits,acc

In [16]:
def get_preds(out):
    preds = torch.argmin(out, dim=0).item()
    return preds, preds

In [17]:
# Define the dataset and the respective loaders
args.train_episodes=50
train_dataset = DataSet( args.image_size, 'train',args.data_path)
train_sampler = EpisodeSampler(train_dataset.label, args.train_episodes,args.num_classes, args.k_shot, args.query, args.unlabel)
trainloader = DataLoader(train_dataset, batch_sampler=train_sampler,shuffle=False, num_workers=8, pin_memory=True)

In [18]:
print("Enter trainloader")
for i,data in enumerate(trainloader):
        print("\nEpisode :",i)
        # create different sets of data from the train loader
        data = data                #cpu()
        targets = torch.arange(args.num_classes).repeat(args.k_shot+args.query+args.unlabel).long()

        #print(data,targets)
    
        support_data = data[:num_support]
        query_data = data[num_support:num_support+num_query]
        unlabel_data = data[num_support+num_query:]

        support_targets = targets[:num_support].numpy() #.cpu()
        print(i,support_targets.shape)
        
        criterion = nn.CrossEntropyLoss(reduction='mean')
        #print(last_layer.parameters())
        best_acc=10
        optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3, momentum=0.9, weight_decay=5e-4)  # weight decay is for L2 regularization.
        print("Start Training")
        for epoch in range(20):
              loss,_,acc=train_loop2(model, None, criterion, optimizer, support_data, support_targets)
              print(epoch,loss,acc)
        torch.cuda.empty_cache()
        gc.collect()
        if(i==0):
               best_acc=acc
               torch.save(model,'Mymodel.pt')
        elif(acc>best_acc):
               best_acc=acc
               torch.save(model,'Mymodel.pt')

Enter trainloader

Episode : 0
0 (640,)
Start Training


C:\Users\kanad\AppData\Local\Temp\ipykernel_14620\801308045.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(inputs[i])
C:\Users\kanad\AppData\Local\Temp\ipykernel_14620\801308045.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits=F.softmax(logits) # Changed here
C:\Users\kanad\AppData\Local\Temp\ipykernel_14620\801308045.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(inputs[i])


0 4.160763943195343 0.9375
1 4.160720925033092 0.625
2 4.16058858782053 1.25
3 4.1606063291430475 1.25
4 4.160524771362543 1.5625
5 4.1604601316154 1.40625
6 4.160555212199688 0.46875


KeyboardInterrupt: 